In [3]:
# Даны два массива: [1, 2, 3, 2, 0] и [5, 1, 2, 7, 3, 2]
# Надо вернуть [1, 2, 2, 3] (порядок неважен)

In [4]:
def common_elements(a, b):
    b_dict = defaultdict(int)  # defaultdict выжил :)
    for el in b:
        b_dict[el] += 1  # я считаю все элементы из b, т.е. типа collections.Counter

    result = []

    for el in a:
        count = b_dict[el]
        if count > 0:  # если какой-то элемент из a встречается в b
            result.append(el)  # то это успех
            b_dict[el] -= 1  # и я "вынимаю" его из b, т.е. уменьшаю его количество на 1

    return result

In [5]:
# Дана строка (возможно, пустая), состоящая из букв A-Z: AAAABBBCCXYZDDDDEEEFFFAAAAAABBBBBBBBBBBBBBBBBBBBBBBBBBBB
# Нужно написать функцию RLE, которая на выходе даст строку вида: A4B3C2XYZD4E3F3A6B28 
# И сгенерирует ошибку, если на вход пришла невалидная строка. 
# Пояснения: Если символ встречается 1 раз, он остается без изменений; Если символ повторяется более 1 раза, к нему добавляется количество повторений.

In [6]:
def convert(s: str) -> str:
    result = []
    last_sym = None  # последний символ, что мы видели
    count = 0  # и сколько мы его видели

    # мы будем идти по строке и записывать в result при смене символа
    for sym in (list(s) + [None]):  # последний None искусственно триггерит посленюю смену символа
        if last_sym and sym != last_sym:  # если случилась смена символа
            if count == 1:
                result.append(last_sym)
            else:
                result.append(last_sym + str(count))
            count = 1  # начнаем запоминать новый символ
            last_sym = sym
        else:  # символ просто повторился
            count += 1  # ну ок, запомнили что символ видели на 1 раз больше

    return ''.join(result)

In [7]:
# Дан массив из нулей и единиц.
# Нужно определить, какой максимальный по длине
# подинтервал единиц можно получить, удалив ровно один элемент массива. [1, 1, 0]

In [10]:
from typing import List
# пример: [0, 0, 1, 1, 0, 1, 1, 0]

def max_ones_length(lst: List[int]) -> int:
    max_ones_length = 0

    # тут мы хотим получить сгруппированные 0 или 1 и их количество:
    subranges = []  # [(0, 2), (1, 2), (0, 1), (1, 2), (0, 1)]
    last_el = None  # последний элемент, который мы просмотрели

    # идём по элементам списка
    for el in lst + [0]:  # [0] - это ручной триггер для обработки последнего элемента

        if last_el != el:  # если произошла смена 0 на 1 или наоборот
            if el == 0:  # если это была смена 1 на 0
              
                # пример: subranges == [(0, 2), (1, 2), (0, 1), (1, 2)]
                # у нас произошла смена 1 на 0, до смены единица шла 2 раза
                # (см последний элемент subranges) - проверяем, вдруг это
                # максимальная длина
                try:
                    last_ones_length = subranges[-1][1]
                    max_ones_length = max(last_ones_length, max_ones_length)
                except IndexError:
                    pass

                # а может если мы удалим один ноль между элементами 1 и 3,
                # то получится более длинная последовательность единиц?
                try:
                    gap_length = subranges[-2][1]
                    if gap_length == 1:
                        combined_ones_length = subranges[-1][1] + subranges[-3][1]
                        max_ones_length = max(combined_ones_length, max_ones_length)
                except IndexError:
                    pass

            # добавляем новый счётчик последовательности в subranges
            subranges.append((el, 1))

        else:
            # увеличиваем счётчик последней последовательности на 1
            subranges[-1] = (el, subranges[-1][1]+1)

        last_el = el

    # костыль, граничное условие
    if len(subranges) == 2 and subranges[1][1] == 1:
        return subranges[0][1] - 1

    return max_ones_length

In [11]:
# Даны даты заезда и отъезда каждого гостя.
# Для каждого гостя дата заезда строго раньше даты отъезда (то есть каждый гость останавливается хотя бы на одну ночь).
# В пределах одного дня считается, что сначала старые гости выезжают, а затем въезжают новые. 
# Найти максимальное число постояльцев, которые одновременно проживали в гостинице (считаем, что измерение количества постояльцев происходит в конце дня).
# sample = [(1, 2), (1, 3), (2, 4), (2, 3), ]

In [12]:
from collections import defaultdict

def max_num_guests(guests: List[tuple]) -> int:
    res = 0
    
    # для каждого дня посчитаем, сколько приехало и сколько отъехало
    arriving = defaultdict(int)
    leaving = defaultdict(int)

    for guest in guests:  # O(n)
        arriving[guest[0]] += 1
        leaving[guest[1]] += 1

    current = 0
    # едем по дням в порядке увеличения, добавлем приехавших и убавляем уехавших,
    # считаем сколько стало
    for day in sorted(set(arriving.keys()).union(set(leaving.keys()))):  # O(n*log(n)) + O(n)
        current -= leaving[day]
        current += arriving[day]

        if current > res:
            res = current

    return res

In [13]:
# Sample Input ["eat", "tea", "tan", "ate", "nat", "bat"]
# Sample Output [ ["ate", "eat", "tea"], ["nat", "tan"], ["bat"] ]
# Т.е. сгруппировать слова по "общим буквам".

In [14]:
def group_words(words: List[str]) -> List[List[str]]:

    groups = defaultdict(list)

    for word in words:  # O(n)
        key = sorted(word)
        groups[key].append(word)

    return [sorted(words) for words in groups.values()]  # O(n*log(n))

In [15]:
# Слияние отрезков:
# Вход: [1, 3] [100, 200] [2, 4] 
# Выход: [1, 4] [100, 200]

In [25]:
from typing import Tuple

def merge(ranges: List[Tuple[int, int]]) -> List[Tuple[int, int]]:

    if not ranges:
        return []

    result_ranges = []
    last_range = None  # последний отрезок, что мы видели

    for rng in sorted(ranges):  # обязательно сортируем

        if not last_range:
            last_range = rng
            continue

            # если начало текущего отрезка меньше конца предыдущего
        if rng[0] <= last_range[1]:
            # расширяем предыдущий отрезок на текущий
            last_range = (last_range[0], max(rng[1], last_range[1]))
            # старый отрезок всё, начинаем новый
        else:
            result_ranges.append(last_range)
            last_range = rng

    else:
        # граничный случай для последнего элемента
        result_ranges.append(last_range)

    return result_ranges

In [28]:
# Дан массив точек с целочисленными координатами (x, y).
# Определить, существует ли вертикальная прямая, делящая точки на 2 симметричных относительно этой прямой множества.
# Note: Для удобства точку можно представлять не как массив [x, y], а как объект {x, y}

In [29]:
class Point:
    pass

In [30]:
from statistics import mean

def is_vertical_symmetry(points: List[Point]) -> bool:

    # сначала найдём вертикальную прямую в середине всех точек
    x_center = mean(p.x for p in points)

    # тут будем хранить точки, для которых пока не нашли пары:
    unmatched_points = defaultdict(int)
    
    for point in points:

        if point.x == x_center:  # если точка на прямой, то она сама себе пара
            continue

        # создадим "брата" - точку, которая симметрична текущей относительно вертикальной прямой
        brother = Point(
            x = x_center * 2 - point.x,
            y = point.y,
        )
        
        # если этот брат есть в unmatched_points, то достаём его оттуда и говорим, что текущая точка сматчилась
        if unmatched_points[brother]:
            unmatched_points[brother] -= 1
            
        # иначе добавляем эту точку в не-сматченные
        else:
            unmatched_points[point] += 1

    return not any(unmatched_points.values())

## Найти наибольшую сумму в дереве

In [32]:
# Найти наибольшую сумму в дереве
res_max = float('-inf')

def findMax(root):
    if not root:
        return 0
    left_value = findMax(root.left)
    right_value = findMax(root.right)
    
    left_max = left_value + root.data
    right_max = right_value + root.data
    all_max = left_value + right_value + root.data
    tmp_max = max(left_max, right_max, all_max)
    if tmp_max > res_max:
        res_max = tmp_max
    return tmp_max